In [ ]:
'''
2. Neural Network Classifier with Keras
Using the multi-label classifier dataset from earlier exercises (categorized-comments.jsonl in the reddit folder), fit a neural network classifier using Keras. Use the code found in chapter 12 of the Applied Text Analysis with Python book as a guideline. Report the accuracy, precision, recall, F1-score, and confusion matrix.
'''

In [ ]:
# Training a multiclass Classifier
# Using Keras to construct a feedforward neural network with an output layer with soft-max activation functions

# Load libraries
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers

#import json library to read data in jsonl file
import json
#import pandas library
import pandas as pd

In [ ]:
#read in the data as a dataframe
filename = "/home/arindam/Documents/mygithub/bu_dsc/data/raw/categorized-comments.jsonl"
with open(filename, 'r') as f:
    jsonl_list = list(f)

list1 = []
for obj in jsonl_list:
    res = json.loads(obj)
    list1.append(res)
    
comments = pd.DataFrame(list1)

#display the first few rows of data
comments.head()
# len(list1)

In [ ]:
#Convert text to lowercase and romove punctuation
#define a function to clean the text
# import the required libraries here
#import regular expressions library
import re

def clean_text(text):
    """
    Remove punctuations and special characters, makes lower case
    Args: text
    Output: text
    """
    text=text.lower() #makes text lowercase
    text=re.sub('\\d|\\W+|_',' ',text) #removes extra white space
    text=re.sub('[^a-zA-Z]'," ", text) #removes any non-alphabetic characters
    return text

In [ ]:
# Cleaning texts in the comments
# Using the transformed column for the model
# Testing the functions

sample_size = 500000
sample_cmnts = comments[:sample_size]
sample_cmnts['cleaned']=sample_cmnts['txt'].apply(clean_text)
# creating a dictionary to replace the string values to numeric
d = {'sports':1,'science_and_technology':2,'video_games':3}
sample_cmnts['ncat'] = sample_cmnts['cat'].map(d)
sample_cmnts.head

In [ ]:
sample_cmnts.ncat.value_counts()

In [ ]:
# Get the target name
from sklearn.model_selection import train_test_split

# Creating the features from the data set
features, target = sample_cmnts.cleaned, sample_cmnts.ncat

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

number_of_features = 2000

tfidf = TfidfVectorizer(max_features = number_of_features, stop_words=stopwords.words('english'))

features = tfidf.fit_transform(features).toarray()
features.shape

In [ ]:
# Make test and training split (30:70)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(features,target, random_state=0, test_size = 0.3)

In [ ]:
# X_train.shape
# X_test.shape
# y_train.shape
# y_test.shape

In [ ]:
# Start Neural Network
network = models.Sequential()

In [ ]:
# Adding fully connected input layer with a ReLU activation function
network.add(layers.Dense(units=100,
                         activation='relu',
                         input_shape=(number_of_features,)))

In [ ]:
# Add fully connected layer with ReLU activation function
network.add(layers.Dense(units=100, activation="relu"))

In [ ]:
# Add fully connected layer with a softmax activation function
network.add(layers.Dense(units=3, activation="softmax"))

In [ ]:
# Compile neural network
network.compile(loss="categorical_crossentropy", # Cross-entropy
                optimizer="rmsprop", # Root Mean Square Propagation
                metrics=["accuracy"]) # Accuracy performance metric

In [ ]:
# Train neural network
history = network.fit(X_train, # Features
                      y_train, # Target
                      epochs=3, # Three epochs
                      verbose=1, # No output
                      batch_size=100, # Number of observations per batch
                      validation_data=(X_test, y_test)) # Test data

In [ ]:
# Showing the accuracy for the last epoch-the final epoch
training_accuracy = history.history["accuracy"]
test_accuracy = history.history["val_accuracy"]

print("Training Accuracy :" ,training_accuracy)
print("Test Accuracy :" ,test_accuracy)

In [ ]:
features = tfidf_matrix
features.todense()

In [ ]:
# get the first vector out (for the first document) 
print(features[0]) 

In [ ]:
# place tf-idf values in a pandas data frame 

# features.T.todense()
# tfidf.get_feature_names()
'''
df = pd.DataFrame(features.T.todense(), index=tfidf.get_feature_names(),columns=[["sam123"]]) 
df1 = df.sort_values(['sam123'], ascending=[True])
df1
'''